In [87]:
import brightway2 as bw
from bw2io import ExcelLCIAImporter
import pandas as pd
from premise import *
from premise_gwp import add_premise_gwp
from pathlib import Path

## Create a new project

In [88]:
PROJECT_NAME = 'iri' # <- insert your project name here
bw.projects.set_current(PROJECT_NAME)

## Install databases

**Biosphere database**

In [ ]:
if 'biosphere3' not in bw.databases:
    bw.bw2setup()

**Ecoinvent v3.8 cut-off system model**

In [ ]:
ECOINVENT_DIR = r'.../ei38/datasets' # <- insert the path to ecoinvent 3.8 here

if 'ecoinvent 3.8 cutoff' in bw.databases:
    pass
else:
    ei38 = bw.SingleOutputEcospold2Importer(ECOINVENT_DIR, 'ecoinvent 3.8 cutoff')
    ei38.apply_strategies()
    ei38.statistics()
    ei38.write_database()

**Prospective databases with premise**

The use of default IAM scenarios included in *premise* requires a decryption key to be requested from the library maintainers. For further details on *premise* see: https://github.com/polca/premise

In [ ]:
DECRYPTION_KEY = 'XXXXXXXXXXXXXXXXX' # <- insert your decryption key here

In [ ]:
ei_future = NewDatabase(
                scenarios=[{"model": "image", "pathway":"SSP2-RCP19", "year":2030},
                           {"model": "image", "pathway":"SSP2-RCP19", "year":2040},
                           {"model": "image", "pathway":"SSP2-RCP19", "year":2050},
                          ],
                source_db="ecoinvent 3.8 cutoff",
                source_version="3.8",
                key=DECRYPTION_KEY)

In [ ]:
ei_future.update_all()

In [ ]:
ei_future.write_db_to_brightway(name=["ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2030",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2040",
                                      "ecoinvent 3.8 cutoff, IMAGE SSP2-RCP19, 2050",
                                     ])

## Create/update LCIA methods

Here we implement some new methods that are not available in the biosphere version compatible with Ecoinvent 3.8, including:

    - IPCC 2021 global warming potentials (GWPs) adapted to premise
    - EF 3.1 for ecotoxicity and human toxicity
    - LANCA v2.5 for land use soil erosion

The following data files are required (to be obtained from the corresponding sources:

    - Methods implementation in Ecoinvent 3.9: "LCIA Implementation 3.9.1.xlsx" (available at https://ecoinvent.org/)
    - Because the name of some elementary flows change from Ecoinvent 3.8 to 3.9, the change report file is also required: "Change Report Annex v3.8 - v3.9.xlsx" (available at https://ecoinvent.org/)
    - 100-year GWPs adapted to premise: "lcia_gwp_100a_w_bio.xlsx" (available at https://github.com/polca/premise_gwp)
    - Caracterization factors from LANCA v2.5 (data file provided in this repository)

In [103]:
# Import LCIA implementation data
METHODS_DIR = Path(r"..\data\LCIA Implementation 3.9.1.xlsx")
CHANGE_REPORT_DIR = Path(r'..\data\Change Report Annex v3.8 - v3.9.xlsx')
PREMISE_GWP_DIR = Path(r'..\data\lcia_gwp_100a_w_bio.xlsx')

lcia_ei_391 = pd.read_excel(METHODS_DIR, sheet_name='CFs')
lcia_change_ei39vs38 = pd.read_excel(CHANGE_REPORT_DIR, sheet_name='LCIA Changes')
premise_gwp = pd.read_excel(PREMISE_GWP_DIR)

In [97]:
# Map biosphere flows
map_bio_keys = {}
for ef in bw.Database('biosphere3'):
    map_bio_keys[(ef['name'], ef['categories'])] = ef.key

**Climate change: IPCC2021 adapted to premise**

Since prospective databases generated with *premise* may involve inventories containing
net negative emission technologies (NETs), it is highly recommended to account for
biogenic CO2 flows as well as CO2 captured from the atmosphere. For further details see: https://github.com/polca/premise_gwp

Premise GWPs are based on IPCC2013. Here we update the method with the GWPs from IPCC2021

In [ ]:
# IPCC 2021 method
ipcc2021_gwp_100a = lcia_ei_391[(lcia_ei_391['Method'] == 'IPCC 2021') & 
                                (lcia_ei_391['Category'] == 'climate change: including SLCFs') & 
                                (lcia_ei_391['Indicator'] == 'global warming potential (GWP100)')]
    
# Adapt GWPs from IPCC 2021 to premise GWPs:
premise_ipcc2021_gwp = dict()
for index, row in premise_gwp.iterrows():
    premise_ipcc2021_gwp.update({(row['name'], row['categories']): row['amount']})

    # Update existing and add new characterization factors based on IPCC2021
    for index, row in ipcc2021_gwp_100a.iterrows():
        ef_info = (row['Name'], row['Compartment'] + '::' + row['Subcompartment'])
        # Update existing elementary flows:
        if ef_info in premise_ipcc2021_gwp:
            premise_ipcc2021_gwp[ef_info] = row['CF']
        else:
           premise_ipcc2021_gwp.update({ef_info: row['CF']})

    # Export to Excel:
    premise_ipcc2021_df = pd.DataFrame(premise_ipcc2021, index=['amount']).T.reset_index()
    premise_ipcc2021_df.columns = ['name', 'categories', 'amount']
    premise_ipcc2021_df.to_excel(gwp_premise_files[indicator][2], index=False)

**Climate change: CO2 emissions**

The carrying capacity for climate change is defined based on the remaining carbon budget. This requires quantifying the life cycle CO2 emissions. We create a new method to quantify this emissions

In [33]:
# Use the new IPCC2021 method to retrieve CO2 flows and characterization factors
ipcc_2021_method = bw.Method(('IPCC 2021', 'climate change: including SLCFs', 'GWP 100a, incl. H and bio CO2'))
gwp = ipcc_2021_method.load()
co2_cf = [i for i in gwp if 'Carbon dioxide' in bw.Database('biosphere3').get(i[0][1])['name']]

In [35]:
[(bw.Database('biosphere3').get(i[0][1])['name'], i[1]) for i in co2_cf]

[('Carbon dioxide, in air', -1),
 ('Carbon dioxide, non-fossil, resource correction', -1),
 ('Carbon dioxide, non-fossil', 1),
 ('Carbon dioxide, non-fossil', 1),
 ('Carbon dioxide, non-fossil', 1),
 ('Carbon dioxide, non-fossil', 1),
 ('Carbon dioxide, fossil', 1),
 ('Carbon dioxide, fossil', 1),
 ('Carbon dioxide, fossil', 1),
 ('Carbon dioxide, fossil', 1),
 ('Carbon dioxide, fossil', 1),
 ('Carbon dioxide, from soil or biomass stock', 1),
 ('Carbon dioxide, from soil or biomass stock', 1),
 ('Carbon dioxide, from soil or biomass stock', 1),
 ('Carbon dioxide, from soil or biomass stock', 1),
 ('Carbon dioxide, from soil or biomass stock', 1),
 ('Carbon dioxide, from soil or biomass stock', 1),
 ('Carbon dioxide, to soil or biomass stock', -1),
 ('Carbon dioxide, to soil or biomass stock', -1),
 ('Carbon dioxide, to soil or biomass stock', -1),
 ('Carbon dioxide, to soil or biomass stock', -1)]

In [37]:
# Write method
method = bw.Method(('IPCC 2021', 'climate change: CO2 emissions'))
metadata = {"unit": 'kg CO2'}
method.register(**metadata)
method.write(co2_cf)
method.process()

**Land use: LANCA method for erosion potential due to land occupation and transformation (to/from)**

In [86]:
lanca_cat = (
             ("LANCA v2.5 - land use", "erosion potential"),
             "kg soil loss",
             "LANCA v2.5 - Characterization Factors for Erosion Potential due to land occupation and transformation (to/from) \
             Available at: https://www.bookshop.fraunhofer.de/buch/LANCA/244600",
             "Soil-Erosion-Potential_CFs_LANCA_v1.5.xlsx"
            )

lanca_method = bw.ExcelLCIAImporter(filepath=Path(str(r"..\data") + f"/{lanca_cat[-1]}"), name=lanca_cat[0], unit=lanca_cat[1], description=lanca_cat[2])

# apply formatting strategies
lanca_method.apply_strategies()

# drop unlinked flows
lanca_method.drop_unlinked()

# write method
lanca_method.write_methods(overwrite=True, verbose=True)

Applying strategy: csv_restore_tuples
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 8 strategies in 0.14 seconds
Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds
Wrote 1 LCIA methods with 135 characterization factors


In [84]:
len(list(lanca_method.unlinked))

0

In [68]:
occupation_not_in = [i['name'] for i in list(lanca_method.unlinked) if 'Transformation, to' in i['name']]
occupation_not_in

['Transformation, to agriculture',
 'Transformation, to agriculture, mosaic',
 'Transformation, to annual crop, fallow',
 'Transformation, to forest, natural',
 'Transformation, to forest, used',
 'Transformation, to grassland',
 'Transformation, to grassland/pasture/meadow',
 'Transformation, to traffic area',
 'Transformation, to unspecified, used',
 'Transformation, to urban',
 'Transformation, to urban, green areas',
 'Transformation, to urban/industrial fallow',
 'Transformation, to water bodies, artificial']

In [69]:
for ii in [i[0] for i in map_bio_keys if 'Transformation, to' in i[0]]:
    if ii not in [i['name'] for i in lanca_method.data[0]['exchanges'] if 'Transformation, to' in i['name']]:
        print(ii)

Transformation, to seabed, drilling and mining
Transformation, to seabed, natural (non-use)
Transformation, to field margin/hedgerow
Transformation, to bare area (non-use)
Transformation, to lake, artificial
Transformation, to arable land, unspecified use
Transformation, to seabed, unspecified
Transformation, to river, natural (non-use)
Transformation, to river, artificial
Transformation, to urban/industrial fallow (non-use)
Transformation, to annual crop, flooded crop
Transformation, to inland waterbody, unspecified
Transformation, to wetland, coastal (non-use)
Transformation, to snow and ice (non-use)
Transformation, to lake, natural (non-use)
Transformation, to unspecified
Transformation, to heterogeneous, agricultural
Transformation, to seabed, infrastructure
Transformation, to urban, green area
Transformation, to cropland fallow (non-use)
